<h1 style='color:#0000ec'>8 &nbsp;Estudo comparativo - Soluções numéricas e analítica</h1>

<h2 style='color:#0000b7' >&nbsp; 8.1 &nbsp;Erro absoluto e percentual</h2>

&nbsp; &nbsp; &nbsp; Esta seção tem como objetivo comparar a solução analítica e a aproximação numérica com a população estimada para 2021 pelo IBGE.

&nbsp; &nbsp; &nbsp; Os resultados obtidos pela solução analítica, pelo método de Euler, Euler modificado e RK4:

<center><b>Aproximação populacional pelo modelo de Malthus</b></center>

| Método | Número de habitantes |
| --- | --- |
| LSODA | 209760.59999083 |
| Euler | 209759.35909435 |
| Euler modificado | 209759.42816025 |
|RK4 |209759.42816037 |
|RK45|  209760.60286601182|

<center>Fonte: O autor</center>

<br>

<center><b>Aproximação populacional pelo modelo de Verhulst</b></center>

| Método | Número de habitantes |
| --- | --- |
| LSODA | 208447.55940518 |
| Euler | 208446.62006659 |
| Euler modificado | 208446.56882837 |
|RK4 |208446.56882864 |
|RK45|  208447.55725928|

<center>Fonte: O autor</center>